In [1]:
from newsapi import NewsApiClient
import json
import pandas as pd
import numpy as np
import re

In [2]:
# /v2/everything
#all_articles = newsapi.get_everything(q='bitcoin',
#                                          from_param='2020-12-01',
#                                          to='2020-12-24',
#                                          language='en',
#                                          sort_by='relevancy',
#                                          page=2)

In [3]:
# # # # NEWS API AUTHENTICATER # # # #
class NewsAuthenticator():

    def authenticate_news_api(self):
        auth = NewsApiClient(api_key='cc3f4e3191f149658f3922e9c47ec1ad')
        return auth

In [13]:
# # # # NEWS API STREAMER # # # #
class ArticleStreamer():
    """
    Class for streaming and processing articles daily
    """
    def __init__(self):
        self.news_authenticator = NewsAuthenticator()    

    def stream_articles(self, query, from_param, to):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        newsapi = self.news_authenticator.authenticate_news_api()
        all_articles = newsapi.get_everything(q=query,
                                          from_param= from_param,
                                          to= to,
                                          language='en')
        
        return all_articles

In [5]:
class ArticleCleaner():
    """
    Functionality for importing and cleaning news articles
    """
    def import_json(self, fetched_json_file):
        # Import json and normalize data
        df = pd.read_json(fetched_json_file)
        norm_articles = pd.json_normalize(df['articles'])
        
        return norm_articles

    def clean_article(self, article):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", article).split())
    
    def create_article(self, articles):
        article = articles[['author', 'title', 'description', 'url', 'source.name', 'publishedAt']]
        outlet = articles[['source.id', 'source.name']]
        
        return article, outlet

In [6]:
# /v2/top-headlines
#top_headlines = newsapi.get_top_headlines(q='bitcoin',
#                                          language='en',
#                                          category='business',
#                                          country='us')

In [7]:
# /v2/sources
#sources = newsapi.get_sources()

In [15]:
if __name__ == '__main__':
    fetched_json_file = './news_articles.json'
    query = 'cryptocurrency'
    from_param = '2020-12-20'
    to = '2021-01-19'
    
    article_cleaner = ArticleCleaner()
    article_streamer = ArticleStreamer()
    
    all_articles = article_streamer.stream_articles(query, from_param, to)
    
    # Export json
    with open(fetched_json_file, 'w') as f:
        json.dump(all_articles, f)
    
    article_data = article_cleaner.import_json(fetched_json_file)
    articles, outlets = article_cleaner.create_article(article_data)
    
    clean_title = np.array([article_cleaner.clean_article(article) for article in articles['title']])
    clean_desc = np.array([article_cleaner.clean_article(article) for article in articles['description']])
    
    articles['clean_title'] = clean_title
    articles['clean_description'] = clean_desc
    
    articles.to_csv(r'.\cleaned_articles_crypto.csv', index=False)
    outlets.to_csv(r'.\outlets.csv', index=False)

<ipython-input-15-116d8d448beb>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['clean_title'] = clean_title
<ipython-input-15-116d8d448beb>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['clean_description'] = clean_desc


In [10]:
article_data

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Kevin Buckland,UPDATE 1-Bitcoin trades near Sunday record of ...,"Bitcoin traded at $33,176 in Asia on Monday, a...",https://uk.reuters.com/article/uk-crypto-curre...,https://static.reuters.com/resources/r/?m=02&d...,2021-01-04T02:47:00Z,FILE PHOTO: A representation of virtual curren...,reuters,Reuters
1,Kevin Buckland,"Bitcoin trading at $32,990, off Sunday record ...","Bitcoin traded at $33,176 in Asia on Monday, a...",https://in.reuters.com/article/crypto-currency...,https://static.reuters.com/resources/r/?m=02&d...,2021-01-04T02:12:00Z,FILE PHOTO: A representation of virtual curren...,reuters,Reuters
2,Kevin Buckland,UPDATE 1-Bitcoin slides more than 5% after top...,"Bitcoin fell more than 5% on Friday, a day aft...",https://www.reuters.com/article/crypto-currenc...,https://static.reuters.com/resources/r/?m=02&d...,2021-01-08T03:17:00Z,FILE PHOTO: A representation of virtual curren...,reuters,Reuters
3,Brendan Hesse,Don't Use Telegram's New 'People Nearby' Feature,Telegram’s new “People Nearby” feature shows a...,https://lifehacker.com/dont-use-telegrams-new-...,https://i.kinja-img.com/gawker-media/image/upl...,2021-01-08T16:00:00Z,Telegrams new People Nearby feature shows a li...,None,Lifehacker.com
4,Timothy B. Lee,The world’s cryptocurrency is now worth more t...,"Bitcoin's anonymous founder, Satoshi Nakamoto,...",https://arstechnica.com/tech-policy/2021/01/th...,https://cdn.arstechnica.net/wp-content/uploads...,2021-01-07T17:30:27Z,1 with 1 posters participating\r\nThe world's ...,ars-technica,Ars Technica
5,Reuters Staff,Bitcoin emergence as 'digital gold' could lift...,Digital currency bitcoin has emerged as a riva...,https://www.reuters.com/article/us-crypto-curr...,https://static.reuters.com/resources/r/?m=02&d...,2021-01-05T13:50:00Z,By Reuters Staff\r\nLONDON (Reuters) - Digital...,reuters,Reuters
6,Reuters Staff,Bitcoin emergence as 'digital gold' could lift...,Digital currency bitcoin has emerged as a riva...,https://in.reuters.com/article/crypto-currenci...,https://static.reuters.com/resources/r/?m=02&d...,2021-01-05T13:54:00Z,By Reuters Staff\r\nLONDON (Reuters) - Digital...,reuters,Reuters
7,Reuters Staff,UPDATE 1-Bitcoin hits one-week low as rising U...,"Cryptocurrencies plunged on Monday, with bitco...",https://www.reuters.com/article/crypto-currenc...,https://static.reuters.com/resources/r/?m=02&d...,2021-01-11T04:20:00Z,By Reuters Staff\r\nFILE PHOTO: A representati...,reuters,Reuters
8,Dan Whateley,"Bitcoin crosses $30,000 for the first time as ...",Summary List PlacementThe price of bitcoin cro...,https://www.businessinsider.com/bitcoin-crosse...,https://i.insider.com/5fe5e0dda6448800181935ea...,2021-01-02T19:46:10Z,"The price of bitcoin crossed $30,000 for the f...",business-insider,Business Insider
9,Isobel Asher Hamilton,Jack Dorsey criticized proposed cryptocurrency...,The Twitter founder's open letter comes three ...,http://feedproxy.google.com/~r/entrepreneur/la...,None,2021-01-05T19:00:38Z,None,google-news,Google News
